In [1]:
import os
import numpy as np
import copy
from math import sqrt
from scipy.linalg import solve_triangular

#### Algoritmo para el cálculo del vector de Householder, nombre de función: $house(\cdot)$

In [2]:
"""
    Función que calcula la proyección de householder
    
    params: x       vector al que se le hará la reflexión householder
                    
    return: Beta    constante utilizada para obtener v
            v       vector que representa la reflexión de householder
"""
def house(x):
    m=len(x)
    norm_2_m=x[1:m].dot(np.transpose(x[1:m]))
    v=np.concatenate((1,x[1:m]), axis=None)
    Beta=0
    if (norm_2_m==0 and x[0]>=0):
        Beta=0
    elif (norm_2_m==0 and x[0]<0):
        Beta=2
    else:
        norm_x=np.sqrt(pow(x[0],2)+norm_2_m)
        if (x[0]<=0):
            v[0]=x[0]-norm_x
        else:
            v[0]=-norm_2_m/(x[0]+norm_x)
        Beta=2*pow(v[0],2)/(norm_2_m+pow(v[0],2))
        v=v/v[0]
    return Beta, v

#### Algoritmo para calcular la factorización $QR$ con reflexiones de Householder

In [3]:
"""
    Función de apoyo para genear matrices aleatorias
                            
    params: renglones       no. de renglones de la matriz
            columnas        no. de renglones de la matriz
            maximo_valor    valor máximo de las entradas de la matriz
            minimo_valor    valor mínimo de las entradas de la matriz
            entero          Indica si las entradas serán enteras (True) o no
            
    return: M               Matriz con numeros al azar
"""
def crea_matriz(renglones,columnas,maximo_valor,minimo_valor,entero=False):
    M=np.zeros((renglones, columnas))
    for i in range(renglones):
        for j in range(columnas):
            if entero:
                M[i][j]=(np.random.rand(1)*(maximo_valor+1-minimo_valor)+minimo_valor)//1
            else:
                M[i][j]=np.random.rand(1)*(maximo_valor-minimo_valor)+minimo_valor
    return M

In [4]:
 """
    Función que genera una matriz que contendrá información escencial de las proyecciones householder
    (vectores v's) y componentes de la matriz triangular superior R, del estilo:
    [r11      r12      r13      r14    ]
    [v_2_(1)  r22      r23      r24    ]
    [v_3_(1)  v_3_(2)  r33      r34    ]
    [v_4_(1)  v_4_(2)  v_4_(3)  r44    ]
    [v_5_(1)  v_5_(2)  v_5_(3)  v_5_(4)]
    
    params: A      Matriz (mxn) de la que se desea obtner factorización QR
            
    return: A_r_v  Matriz (mxn) con la información escencial (es igual a la matriz R, pero en lugar de tener ceros
                   en la parte inferior, contiene info de los vectores householder que serán útiles para
                   futuros cálculos, que entre otros están el calcular la matriz ortonormal Q)
"""
def QR(A):
    m=A.shape[0]
    n=A.shape[1]
    A_r_v=copy.copy(A)
    for j in range(n):
        beta, v=house(A_r_v[j:m,j])
        A_r_v[j:m,j:n]=A_r_v[j:m,j:n]-beta*(np.outer(v,v)@A_r_v[j:m,j:n])
        A_r_v[(j+1):m,j]=v[1:(m-j)]
    return A_r_v

In [5]:
"""
    Función que calcula el producto matricial de Q_transpuesta por una matriz dada C
                            
    params: A_r_v   Matriz (mxn) con la info escencial
            C       Matriz (mxp) (si se pasa por ejemplo C=Identidad (mxm) la funcion devolverá Q)

    return: M       Matriz con numero al azar
"""
def QT_C(A_r_v,C):
    m=A_r_v.shape[0]
    n=A_r_v.shape[1]
    QT_por_C=np.eye(m)
    for j in range(n-1,-1,-1):
        v=np.concatenate((1,A_r_v[(j+1):m,j]), axis=None)
        beta=2/(1+A_r_v[(j+1):m,j].dot(A_r_v[(j+1):m,j]))
        QT_por_C[j:m,j:m]=C[j:m,j:m]-beta*np.outer(v,v)@C[j:m,j:m]
    return QT_por_C

In [6]:
"""
    Función que calcula la matriz Qj (en el proceso de obtención de factorización QR se van obteniendo n Qj's,
    que si se multiplican todas da por resultado Q=Q1*Q2*...*Qn)
                            
    params: A_r_v   Matriz (mxn) con la info escencial
            C       Matriz (mxp) (si se pasa por ejemplo C=Identidad (mxm) la funcion devolverá Q)

    return: Qj      Matriz Q de la j-esima iteración del proceso iterativo de factorización QR
"""
def Q_j(A_r_v,j):
    m=A_r_v.shape[0]
    n=A_r_v.shape[1]
    Qj=np.eye(m)
    v=np.concatenate((1,A_r_v[(j+1):m,j]), axis=None)
    beta=2/(1+A_r_v[(j+1):m,j].dot(A_r_v[(j+1):m,j]))
    Qj[j:m,j:m]=np.eye(m-j)-beta*np.outer(v,v)
    return Qj

In [7]:
"""
    Función que obtiene la solución de un sistema de ecuaciones lineala (SEL) con n ecuaciones y n incognitas
            
    params: A   Matriz (nxn) que representa los coeficientas de las ecuaciones
            b   vector (nx1) constantes del sistema

    return: x   vector que satisface (Ax=b)
"""
    #FALTA CONTEMPLAR POSIBLES PROBLEMAS/SOLUCIONES PARA LOS DIFERENTES TIPOS DE SEL (SIN SOLUCION, MULTIPLES SOLUCIONES,
    #SOLUCION ÚNICA). INCORPORAR ESTO PARA EVITAR ERRORES FATALES/DESBORDAMIENTO/ETC  
def Solucion_SEL_QR_nxn(A,b):
    A_r_v=QR(A)
    m=A_r_v.shape[0]
    n=A_r_v.shape[0]
    Q=np.eye(m)
    R=copy.copy(A)
    for j in range(n):
        Qj=Q_j(A_r_v,j)
        Q=Q@Qj
        R=Q_j(A_r_v,j)@R
    b_prima=np.transpose(Q)@b
    x = solve_triangular(R, np.transpose(Q)@b)
    return x

In [8]:
#ALGORITMO SIMPLE DE SUSTITUCION HACIA ATRAS CON UNA MATRIZ TRIANGULAR SUPERIOR (SOLO HAY QUE SUSTITUIR ITERATIVAMENTE)

def Solucion_SHA_triangular_superior_nxn(R,b_prima):
    return x

In [9]:
#PRUEBA DE SOLUCIONES

In [10]:
#Generar una matriz de 5
m=5
n=3
A=np.round(crea_matriz(m,n,6,-6,False),2)
#Prueba también definiendo una matriz entrada por entrada, solo asegurate que sus entradas sean
#tipo flotantes/dobles (pues si son de tipo enteras arrastrarán errores de redondeo considerables)

In [11]:
A

array([[-2.68, -2.75, -5.54],
       [ 2.29,  5.96, -5.68],
       [-0.3 ,  1.23, -0.27],
       [ 4.04, -5.07, -3.49],
       [-2.59, -0.1 , -1.92]])

In [12]:
A_r_v = QR(A)
np.round(A_r_v,4)

array([[ 5.9621,  0.0714, -1.2086],
       [-0.265 ,  8.3849, -0.1166],
       [ 0.0347, -0.4186,  8.7988],
       [-0.4675,  2.0138, -0.6822],
       [ 0.2997, -0.235 ,  0.1875]])

In [13]:
#Obtención de Q con la función QT_C
Q=np.transpose(QT_C(A_r_v,np.eye(m)))
np.round(Q,4)

array([[-0.4495,  0.3841, -0.0503,  0.6776, -0.4344],
       [ 0.3841,  0.8982,  0.0133, -0.1796,  0.1151],
       [-0.0503,  0.0133,  0.9983,  0.0235, -0.0151],
       [ 0.6776, -0.1796,  0.0235,  0.6832,  0.2031],
       [-0.4344,  0.1151, -0.0151,  0.2031,  0.8698]])

In [14]:
#Así obtenemos R
R=np.transpose(Q)@A
np.round(R,4)

array([[ 5.9621,  0.0714, -1.2086],
       [-0.    ,  5.2124, -6.8277],
       [ 0.    ,  1.3279, -0.1196],
       [-0.    , -6.3889, -5.5148],
       [ 0.    ,  0.7456, -0.6219]])

In [15]:
#Ejemplo ilustrativo de cómo obtener Q y R, visualizando cada iteración
Q=np.eye(m)
R=copy.copy(A)
for j in range(n):
    Qj=Q_j(A_r_v,j)
    print('\nQ',j,':\n',np.round(Qj,4))
    Q=Q@Qj
    R=Q_j(A_r_v,j)@R
    print('\nAplicando las Qjs a A por la izquierda para obtener R, iteracion ',j,':\n',np.round(R,4))

print('\n\n\nResultados finales:')
print('\nR es el resultado de multiplicar todas las Qjs a A\n',np.round(R,4))
print('\nQ es el resultado de multiplicar todas las Qjs y transponer:\n',np.round(Q,4))


Q 0 :
 [[-0.4495  0.3841 -0.0503  0.6776 -0.4344]
 [ 0.3841  0.8982  0.0133 -0.1796  0.1151]
 [-0.0503  0.0133  0.9983  0.0235 -0.0151]
 [ 0.6776 -0.1796  0.0235  0.6832  0.2031]
 [-0.4344  0.1151 -0.0151  0.2031  0.8698]]

Aplicando las Qjs a A por la izquierda para obtener R, iteracion  0 :
 [[ 5.9621  0.0714 -1.2086]
 [-0.      5.2124 -6.8277]
 [ 0.      1.3279 -0.1196]
 [-0.     -6.3889 -5.5148]
 [ 0.      0.7456 -0.6219]]

Q 1 :
 [[ 1.      0.      0.      0.      0.    ]
 [ 0.      0.6216  0.1584 -0.762   0.0889]
 [ 0.      0.1584  0.9337  0.3189 -0.0372]
 [ 0.     -0.762   0.3189 -0.5345  0.1791]
 [ 0.      0.0889 -0.0372  0.1791  0.9791]]

Aplicando las Qjs a A por la izquierda para obtener R, iteracion  1 :
 [[ 5.9621  0.0714 -1.2086]
 [ 0.      8.3849 -0.1166]
 [-0.      0.     -2.9288]
 [ 0.     -0.      8.0003]
 [ 0.      0.     -2.1991]]

Q 2 :
 [[ 1.      0.      0.      0.      0.    ]
 [ 0.      1.      0.      0.      0.    ]
 [ 0.      0.     -0.3329  0.9093 -0.2499]

In [16]:
#Se prueba un ejemplo más grande con 10^4 entradas
m=125
n=80
A=np.round(crea_matriz(m,n,10,-10,False),2)
A_r_v = QR(A)
A_r_v

array([[ 6.37876555e+01,  6.67270487e+00,  2.85838221e+00, ...,
         2.50762156e+00, -8.86426214e+00, -2.16130847e+00],
       [ 7.11767191e-02,  6.27367987e+01, -2.12117360e+00, ...,
         9.93024500e+00,  1.44170921e+01,  6.08702200e+00],
       [ 4.65592072e-02, -1.10799680e-01,  6.42716576e+01, ...,
        -3.02921186e+00, -6.34915157e+00,  6.99613162e+00],
       ...,
       [ 1.41104724e-01, -1.19299411e-01,  1.00659903e-01, ...,
         3.08204820e-01,  1.44105746e-01, -1.05177293e-01],
       [ 1.28260805e-01,  1.04911219e-01, -1.23458631e-01, ...,
         1.79489114e-01, -1.15822139e-01,  3.19862058e-01],
       [-1.59478664e-01, -9.86021008e-04, -1.42224485e-01, ...,
         2.08334481e-02, -1.93517644e-01,  6.93193649e-02]])

#### Eliminación por bloques

In [17]:
def bloques(A, b=False, n1=False, n2=False):
    
    """
    Esta es la función para la creación de bloques usando un arreglo de numpy
    """

    # Primero definimos el n
    m,n = A.shape

    # Condiciones de A
    # Si no se dan los n deseados, se intentan hacer los bloques casi iguales
    if  not (n1&n2):
        n1 = n//2
        n2 = n - n1
    # Los bloques deben cumplir la condicion de tamaño
    elif n1+n1 != n:
        sys.exit('n1 + n2 debe ser igual a n')
    else:
        None

    # Condiciones de b
    if  b is False:
        b1 = None
        b2 = None
        print('condicion1')
    elif len(b) == m:
        b1 = b[:n1]
        b2 = b[n1:m]
    else:
        sys.exit('los renglones de A y b deben ser del mismo tamaño')


    A11 = A[:n1,:n1]
    A12 = A[:n1,n1:n]
    A21 = A[n1:m,:n1]
    A22 = A[n1:m,n1:n]

    return A11,A12,A21,A22,b1,b2

In [18]:
def eliminacion_bloques(A,b):

    ## Sean A y A11 no singulares

    if np.linalg.det(A)==0:
        sys.exit('A debe ser no singular')

    A11,A12,A21,A22,b1,b2 = bloques(A,b)

    if np.linalg.det(A11)==0:
        ys.exit('A11 debe ser no singular')

    ## 1. Calcular A11^{-1}A12 y A11^{-1}b1 teniendo cuidado en no calcular la inversa sino un sistema de ecuaciones lineales
    ## Aquí se debe usar el método QR una vez que esté desarrollado

    ## Definimos y = A11^{-1}b1, por tanto A11y=b1. Resolviendo el sistema anterior para 11y:
    y = Solucion_SEL_QR_nxn(A11,b1)
    #y = np.linalg.solve(A11,b1)

    ## Definimos Y = A11^{-1}A12
    Y = Solucion_SEL_QR_nxn(A11,A12)
    #Y = np.linalg.solve(A11,A12)

    ## 2. Calcular el complemento de Schur del bloque A11 en A. Calcular b_hat
    S = A22 - A21@Y
    b_h = b2 - A21@y

    ## 3. Resolver Sx2 = b_hat
    x2 = Solucion_SEL_QR_nxn(S,b_h)
    #x2 = np.linalg.solve(S,b_h)

    ## 4. Resolver A11x1 = b1-A12X2
    x1 = Solucion_SEL_QR_nxn(A11,b1-A12@x2)
    #x1 = np.linalg.solve(A11,b1-A12@x2)

    return np.concatenate((x1,x2), axis=0)

In [19]:
m=6
n=6
A=np.round(crea_matriz(m,n,6,-6,False),2)
b = np.round(crea_matriz(m,1,6,-6,False),2)

In [20]:
print("A:",A)
print("b:",b)

A: [[-0.3  -5.85 -2.99  2.46 -1.58  2.1 ]
 [-5.43 -3.49  0.16  2.46 -3.64  2.05]
 [-0.67 -1.39 -5.93 -4.49  5.79  2.23]
 [ 5.24 -4.77  2.97 -0.78  4.36 -5.01]
 [-5.82 -4.98 -5.89 -0.75 -2.72 -2.42]
 [-4.72  4.82  4.21 -0.79 -4.77  0.51]]
b: [[-0.05]
 [-3.27]
 [ 0.94]
 [ 3.2 ]
 [-5.28]
 [-4.26]]


##### Primero resolvamos el sistema de ecuaciones usando la paquetería de numpy para comparar.

In [21]:
np.linalg.solve(A,b)

array([[0.48783479],
       [0.0521657 ],
       [0.03661165],
       [0.34027567],
       [0.45938243],
       [0.19034988]])

##### Ahora usemos la factorización QR

In [22]:
Solucion_SEL_QR_nxn(A,b)

array([[0.48783479],
       [0.0521657 ],
       [0.03661165],
       [0.34027567],
       [0.45938243],
       [0.19034988]])

##### Por último usemos eliminación por bloques con QR

In [23]:
eliminacion_bloques(A,b)

array([[0.48783479],
       [0.0521657 ],
       [0.03661165],
       [0.34027567],
       [0.45938243],
       [0.19034988]])